In [1]:
import gym_duckietown as gd

DEBUG:commons:version: 6.2.4 *
DEBUG:typing:version: 6.2.3
DEBUG:duckietown_world:duckietown-world version 6.2.38 path /root/.pyenv/versions/3.8.0/lib/python3.8/site-packages
DEBUG:geometry:PyGeometry-z6 version 2.1.4 path /root/.pyenv/versions/3.8.0/lib/python3.8/site-packages
DEBUG:aido_schemas:aido-protocols version 6.0.59 path /root/.pyenv/versions/3.8.0/lib/python3.8/site-packages
DEBUG:nodes:version 6.2.13 path /root/.pyenv/versions/3.8.0/lib/python3.8/site-packages pyparsing 3.0.6
DEBUG:gym-duckietown:gym-duckietown version 6.1.31 path /root/mnt/final-task/env/src



{'audio': ('xaudio2', 'directsound', 'openal', 'pulse', 'silent'), 'debug_font': False, 'debug_gl': True, 'debug_gl_trace': False, 'debug_gl_trace_args': False, 'debug_graphics_batch': False, 'debug_lib': False, 'debug_media': False, 'debug_texture': False, 'debug_trace': False, 'debug_trace_args': False, 'debug_trace_depth': 1, 'debug_trace_flush': True, 'debug_win32': False, 'debug_x11': False, 'graphics_vbo': True, 'shadow_window': True, 'vsync': None, 'xsync': True, 'xlib_fullscreen_override_redirect': False, 'darwin_cocoa': True, 'search_local_libs': True, 'advanced_font_features': False, 'headless': False, 'headless_device': 0, 'win32_disable_shaping': False}


In [3]:
from utils.env import launch_env
from utils.wrappers import NormalizeWrapper, ImgWrapper, DtRewardWrapper, ActionWrapper, ResizeWrapper
import matplotlib.pyplot as plt

マップ一覧
- 4way.yaml
- 4way_bordered.yaml
- Caltech_loop01.yaml
- ETHZ_autolab_fast_track.yaml
- ETHZ_autolab_technical_track.yaml
- ETHZ_autolab_technical_track_bordered.yaml
- ETHZ_loop.yaml
- ETHZ_loop_bordered.yaml
- ETH_intersection_map.yaml
- ETH_large_intersect.yaml
- ETH_large_loop.yaml
- ETH_small_intersect.yaml
- ETH_small_loop.yaml
- ETH_small_loop_1.yaml
- ETH_small_loop_2.yaml
- ETH_small_loop_2_bordered.yaml
- ETH_small_loop_3.yaml
- ETU_autolab_track.yaml
- MOOC_modcon.yaml
- Montreal_loop.yaml
- TTIC_large_loop.yaml
- TTIC_loop.yaml
- TTIC_ripltown.yaml
- calibration_map_ext.yaml
- calibration_map_int.yaml
- experiment_loop.yaml
- field1.yaml
- loop_dyn_duckiebots.yaml
- loop_empty.yaml
- loop_obstacles.yaml
- loop_pedestrians.yaml
- regress_4way_adam.yaml
- regress_4way_drivable.yaml
- robotarium1.yaml
- robotarium2.yaml
- small_loop.yaml
- small_loop_bordered.yaml
- small_loop_cw.yaml
- straight_road.yaml
- straight_road_down.yaml
- udem1.yaml
- udem1_empty.yaml
- zigzag_dists.yaml
- zigzag_dists_bordered.yaml


In [4]:
env = launch_env(map_name="loop_pedestrians")
print("Initialized environment")

# Wrappers
env = ResizeWrapper(env)
env = NormalizeWrapper(env)
env = ImgWrapper(env)  # to make the images from 160x120x3 into 3x160x120
env = ActionWrapper(env)
env = DtRewardWrapper(env)
print("Initialized Wrappers")
# 処理後の観測
state_dim = env.observation_space.shape
print(f"state_dim: {state_dim}")
# 行動次元（２つの車輪のトルク[-1,1]）
action_dim = env.action_space.shape[0]
print(f"action_dim: {action_dim}")
# 最大トルク
max_action = float(env.action_space.high[0])
print(f"max_action: {max_action}")

INFO:gym-duckietown:Information about the graphics card:
 pyglet_version: 1.5.23
    information: dict[4]
                 │ vendor: Mesa/X.org
                 │ renderer: llvmpipe (LLVM 12.0.0, 256 bits)
                 │ version: 3.1 Mesa 21.2.6
                 │ shading-language-version: 1.40
  nvidia_around: True
INFO:duckietown_world: data: /root/.pyenv/versions/3.8.0/lib/python3.8/site-packages/duckietown_world/data
DEBUG:gym-duckietown:loading map file "/root/.pyenv/versions/3.8.0/lib/python3.8/site-packages/duckietown_world/data/gd1/maps/loop_pedestrians.yaml"
INFO:gym-duckietown:done
DEBUG:gym-duckietown:loading mesh 'duckiebot' from file_path '/root/.pyenv/versions/3.8.0/lib/python3.8/site-packages/duckietown_world/data/gd1/meshes/duckiebot/duckiebot.obj'
DEBUG:gym-duckietown:loading materials from /root/.pyenv/versions/3.8.0/lib/python3.8/site-packages/duckietown_world/data/gd1/meshes/duckiebot/duckiebot.mtl
DEBUG:gym-duckietown:loading texture: /root/.pyenv/versions/3.8.

Initialized environment
Initialized Wrappers
state_dim: (3, 120, 160)
action_dim: 2
max_action: 1.0


In [4]:
import argparse

parser = argparse.ArgumentParser()

# DDPG Args
parser.add_argument("--seed", default=0, type=int)  # Sets Gym, PyTorch and Numpy seeds
parser.add_argument(
    "--start_timesteps", default=1e4, type=int
)  # How many time steps purely random policy is run for
parser.add_argument("--eval_freq", default=5e3, type=float)  # How often (time steps) we evaluate
parser.add_argument("--max_timesteps", default=1e6, type=float)  # Max time steps to run environment for
parser.add_argument("--save_models", action="store_true", default=True)  # Whether or not models are saved
parser.add_argument("--expl_noise", default=0.1, type=float)  # Std of Gaussian exploration noise
parser.add_argument("--batch_size", default=32, type=int)  # Batch size for both actor and critic
parser.add_argument("--discount", default=0.99, type=float)  # Discount factor
parser.add_argument("--tau", default=0.005, type=float)  # Target network update rate
parser.add_argument(
    "--policy_noise", default=0.2, type=float
)  # Noise added to target policy during critic update
parser.add_argument("--noise_clip", default=0.5, type=float)  # Range to clip target policy noise
parser.add_argument("--policy_freq", default=2, type=int)  # Frequency of delayed policy updates
parser.add_argument("--env_timesteps", default=500, type=int)  # Frequency of delayed policy updates
parser.add_argument(
    "--replay_buffer_max_size", default=10000, type=int
)  # Maximum number of steps to keep in the replay buffer
parser.add_argument("--model-dir", type=str, default="reinforcement/pytorch/models/")


_StoreAction(option_strings=['--model-dir'], dest='model_dir', nargs=None, const=None, default='reinforcement/pytorch/models/', type=<class 'str'>, choices=None, help=None, metavar=None)

In [5]:
from env.learning.train_reinforcement import _train

_train(parser.parse_args())

AttributeError: 'function' object has no attribute 'model_dir'